<a href="https://colab.research.google.com/github/EdgarAgOr/FAP_Cheminformatic_PET/blob/main/2_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comenzaremos el pretrata de la Base de datos, primero hay que importarla y descargar la paqueteria necesaria

In [ ]:
! pip install -q rdkit

! pip install -q molvs

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import rdMolDescriptors
from molvs.standardize import Standardizer
from molvs.charge import Uncharger, Reionizer
from molvs.fragment import LargestFragmentChooser
from molvs.tautomer import TautomerCanonicalizer
from rdkit.Chem.rdmolops import GetFormalCharge, RemoveStereochemistry

# Para leer un documento csv que hayamos guardado en la compu, ojo que debemos cargarlos en la sesion de colab en el folder

In [ ]:
df = pd.read_csv ("/content/FAP_Bsite_Sin_metales.csv")

In [ ]:
df.columns

Index(['BindingDB_id', 'Ligand InChI', 'Ligand InChI Key',
       'BindingDB MonomerID', 'BindingDB Ligand Name',
       'Target Source Organism According to Curator or DataSource', 'IC50_nM',
       'DataSource', 'Patent Number', 'ChEMBL ID of Ligand',
       'DrugBank ID of Ligand', 'IUPHAR_GRAC ID of Ligand',
       'KEGG ID of Ligand', 'ZINC ID of Ligand',
       'Number of Protein Chains in Target (bigger than 1 implies a multichain complex)',
       'Smiles'],
      dtype='object')

# Comenzaremos con el curado de los datos

In [ ]:
# Vamos a eliminar los compuestos que no tienen reportado un valor numerico de IC50

df2 = df[df.IC50_nM.notna()]

df2

,BindingDB_id,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Source Organism According to Curator or DataSource,IC50_nM,DataSource,Patent Number,ChEMBL ID of Ligand,DrugBank ID of Ligand,IUPHAR_GRAC ID of Ligand,KEGG ID of Ligand,ZINC ID of Ligand,Number of Protein Chains in Target (bigger than 1 implies a multichain complex),Smiles
0,1297087,InChI=1S/C28H50BN7O11/c1-19(2)26(27(44)30-20(3...,PSEWGMXEQWDVLO-SWYRRKHMSA-N,609678.0,"DOTA-Val-DAla- boroPro::US11707539, Compound 6508",Homo sapiens,62.0,US Patent,US11707539,NaN,NaN,NaN,NaN,NaN,1.0,CC(C)[C@H](C(N[C@@H](C)C(N1[C@@H](B(O)O)CCC1)=...
1,1297088,InChI=1S/C28H50BN7O11.Cu/c1-19(2)26(27(44)30-2...,NARBUWFYRBGSCK-GZOVSDKFSA-L,609716.0,"DOTA[(Cu(II)]- Val-D-Ala- boroPro::US11707539,...",Homo sapiens,26.2,US Patent,US11707539,NaN,NaN,NaN,NaN,NaN,1.0,OB(O)[C@H]1CCCN1C(=O)[C@H](C)NC(=O)[C@H](NC(=O...
2,1297089,InChI=1S/C26H46BN7O12/c1-18(26(44)34-4-2-3-20(...,WDEXMCKZAHCKKS-AABGKKOBSA-N,609575.0,"DOTA-Ser-D- Ala-boroPro::US11707539, Compound ...",Homo sapiens,186.0,US Patent,US11707539,NaN,NaN,NaN,NaN,NaN,1.0,C[C@@H](C(N1[C@@H](B(O)O)CCC1)=O)NC([C@@H](CO)...
3,1297090,InChI=1S/C26H46BN7O12.Cu/c1-18(26(44)34-4-2-3-...,OGDGNOAMVPNAQL-SFZWVAMRSA-L,609657.0,"DOTA[(Cu(II)]- Ser-D-Ala- boroPro::US11707539,...",Homo sapiens,128.0,US Patent,US11707539,NaN,NaN,NaN,NaN,NaN,1.0,OB(O)[C@H]1CCCN1C(=O)[C@H](C)NC(=O)[C@H](NC(=O...
4,1297091,InChI=1S/C26H46BN7O11/c1-18(25(42)29-19(2)26(4...,YBPNBOBNLPXOID-ZCNNSNEGSA-N,609724.0,"DOTA-Ala-D- Ala-boroPro::US11707539, Compound ...",Homo sapiens,71.7,US Patent,US11707539,NaN,NaN,NaN,NaN,NaN,1.0,C[C@H](C(N[C@@H](C)C(N1[C@@H](B(O)O)CCC1)=O)=O...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237,1297080,InChI=1S/C26H46BN7O11.Gd/c1-18(25(42)29-19(2)2...,RUTYVUBJBFQIQM-NJBQKBHSSA-K,609727.0,DOTA[(Gd(III)]- Ala-D-Ala- boroPro::US11707539...,Homo sapiens,428.0,US Patent,US11707539,NaN,NaN,NaN,NaN,NaN,1.0,OB(O)[C@H]1CCCN1C(=O)[C@H](C)NC(=O)[C@@H](C)NC...
1238,1297081,InChI=1S/C29H46BN9O11.Gd/c1-20(29(48)39-6-2-3-...,FELKRZOKZGKLSB-LBPAWUGGSA-K,609728.0,DOTA[Gd(III)]- HyNic-D-Ala- boroPro::US1170753...,Homo sapiens,450.0,US Patent,US11707539,NaN,NaN,NaN,NaN,NaN,1.0,OB(O)[C@H]1CCCN1C(=O)[C@H](C)NC(=O)c1ccc(NNC(=...
1239,1297082,InChI=1S/C30H47BN8O11.Gd/c1-21(30(48)39-8-2-3-...,ZHPLOOABVYXWII-WKDBURHASA-K,609729.0,DOTA[(Gd(III)]- HyBz-D-Ala- boroPro::US1170753...,Homo sapiens,68.0,US Patent,US11707539,NaN,NaN,NaN,NaN,NaN,1.0,OB(O)[C@H]1CCCN1C(=O)[C@H](C)NC(=O)c1ccc(cc1)N...
1240,1297083,InChI=1S/C23H41BN6O10/c1-17(23(38)30-4-2-3-18(...,FWQRSNVJTLURHN-MSOLQXFVSA-N,609730.0,"US11707539, Compound 4535",Homo sapiens,290.0,US Patent,US11707539,NaN,NaN,NaN,NaN,NaN,1.0,C[C@@H](C(N1[C@@H](B(O)O)CCC1)=O)NC(CN1CCN(CC(...


# Para este ejemplo vamos a realizar una nueva categorizacion de los datos y los dividiremos entre
# Activos, aquellos que tengan un valor de IC50 <= 1,000,000 nM. (1000 uM)
# Inactivos aquellos que tengan un valor de IC50 >= 10,000,000 nM. (10,000 uM)
# Los demas seran intermedios

In [ ]:
# Con esto generamos una nueva lista que se llamara 'bioactivity_class
bioactivity_class = []

for i in df2.IC50_nM:
    if float(i) >= 10000000:
        bioactivity_class.append ('Inactive')
    elif float (i) <= 1000000:
        bioactivity_class.append ('Active')
    else:
        bioactivity_class.append ('Intermediate')

In [ ]:
bioactivity_class

['Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',
 'Active',

In [ ]:
df2.BindingDB_id

,BindingDB_id
0,1297087
1,1297088
2,1297089
3,1297090
4,1297091
...,...
1237,1297080
1238,1297081
1239,1297082
1240,1297083


In [ ]:
mol_cid = []

for i in df2.BindingDB_id:
    mol_cid.append (i)

In [ ]:
mol_cid

['1297087',
 '1297088',
 '1297089',
 '1297090',
 '1297091',
 '1297092',
 '1297093',
 '1297094',
 '1297095',
 '1297096',
 '1297099',
 '1297100',
 '1297103',
 '1297106',
 '1297109',
 '1297110',
 '1297112',
 '1297114',
 '1297116',
 '1297119',
 '1297122',
 '1297125',
 '1297126',
 '1297128',
 '1297129',
 '1297130',
 '1297131',
 '1297132',
 '1297133',
 '1297134',
 '1297135',
 '1297136',
 '1297137',
 '1297138',
 '1297139',
 '1297140',
 '1297141',
 '1297142',
 '1297143',
 '1297144',
 '1297145',
 '1297146',
 '1297147',
 '1297150',
 '1297153',
 '1297155',
 '1297156',
 '1297157',
 '1297158',
 '1297159',
 '1297161',
 '1297163',
 '1297166',
 '1297169',
 '1297172',
 '1297173',
 '1297175',
 '1297177',
 '1297178',
 '1297180',
 '1297182',
 '1297183',
 '1297186',
 '1297187',
 '1297188',
 '1297189',
 '1297192',
 '1297193',
 '1297195',
 '1297196',
 '1297197',
 '1297199',
 '1297201',
 '1297202',
 '1297204',
 '1297207',
 '50162926',
 '50852829',
 '1297053',
 '1297054',
 '1297055',
 '1297056',
 '1297210',
 '

In [ ]:
canonical_smiles = []

for i in df2.Smiles:
    canonical_smiles.append (i)

In [ ]:
standard_value = []

for i in df2.IC50_nM:
    standard_value.append (i)

# Aqui vamos a combinar las listas hechas en las lineas anteriores en un nuevo DF

In [ ]:
selection = ['BindingDB_id', 'Smiles','IC50_nM']

df3 = df2 [selection]

In [ ]:
df3

,BindingDB_id,Smiles,IC50_nM
0,1297087,CC(C)[C@H](C(N[C@@H](C)C(N1[C@@H](B(O)O)CCC1)=...,62.0
1,1297088,OB(O)[C@H]1CCCN1C(=O)[C@H](C)NC(=O)[C@H](NC(=O...,26.2
2,1297089,C[C@@H](C(N1[C@@H](B(O)O)CCC1)=O)NC([C@@H](CO)...,186.0
3,1297090,OB(O)[C@H]1CCCN1C(=O)[C@H](C)NC(=O)[C@H](NC(=O...,128.0
4,1297091,C[C@H](C(N[C@@H](C)C(N1[C@@H](B(O)O)CCC1)=O)=O...,71.7
...,...,...,...
1237,1297080,OB(O)[C@H]1CCCN1C(=O)[C@H](C)NC(=O)[C@@H](C)NC...,428.0
1238,1297081,OB(O)[C@H]1CCCN1C(=O)[C@H](C)NC(=O)c1ccc(NNC(=...,450.0
1239,1297082,OB(O)[C@H]1CCCN1C(=O)[C@H](C)NC(=O)c1ccc(cc1)N...,68.0
1240,1297083,C[C@@H](C(N1[C@@H](B(O)O)CCC1)=O)NC(CN1CCN(CC(...,290.0


In [ ]:
# Vamos a anexar a esta tabla la columna de bioactivity

data_tuples = list (zip(mol_cid, canonical_smiles, bioactivity_class, standard_value,DataSource ))

df3 = pd.DataFrame (data_tuples , columns = ['BindingDB_id', 'Smiles', 'bioactivity_class', 'IC50_nM','DataSource' ])

NameError: name 'DataSource' is not defined

In [ ]:
df3

In [ ]:
# Vamos a eliminar los compuestos que tengan un valor de " Intermedio" en bioactivity_class

df3_class = df3 [df3.bioactivity_class != "Intermediate"]

In [ ]:
df3_class

In [ ]:
# Vamos a salvar este DF "preprocesado"

df3_class.to_csv ("Bioactividad_FAPBSITE_preprocesada_data.csv", index= False)